In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd ../content/drive/My Drive/project2/Facenet-Real-time-face-recognition-using-deep-learning-Tensorflow-master

/content/drive/My Drive/project2/Facenet-Real-time-face-recognition-using-deep-learning-Tensorflow-master


In [0]:
ls

abc1.jpg            detect_face.py          model/         test.py
abc.jpg             detect_face.pyc         npy/           train_img/
akshay_mov.mp4      facenet.py              pre_img/       train_main.py
class/              facenet.pyc             preprocess.py  Untitled0.ipynb
classifier.py       identify_face_image.py  __pycache__/   Untitled.ipynb
data_preprocess.py  identify_face_video.py  README.md


In [0]:
!python train_main.py

Training Start
2019-05-15 16:17:54.499054: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-15 16:17:54.499352: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x26f9e40 executing computations on platform Host. Devices:
2019-05-15 16:17:54.499416: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-15 16:17:54.637573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-15 16:17:54.638198: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x26f98c0 executing computations on platform CUDA. Devices:
2019-05-15 16:17:54.638265: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-15 16:17:54.638670: I tensorflow/core/common_runtime/gpu/gpu_device.cc

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from scipy import misc
import cv2
import numpy as np
import facenet
import detect_face
import os
import time
import pickle
import sys

sess = None
pnet= None
rnet= None
onet= None
embedding_size= None
images_placeholder= None
phase_train_placeholder= None
embeddings= None
model= None
HumanNames= None

def Init(): 
 global sess,pnet,rnet,onet,embedding_size,images_placeholder,embeddings,model,HumanNames,phase_train_placeholder
 modeldir = 'model/20170511-185253.pb'
 classifier_filename = 'class/classifier.pkl'
 npy='npy'
 train_img="train_img"

 with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, npy)
        HumanNames = os.listdir(train_img)
        HumanNames.sort()

        print('Loading feature extraction model')
        facenet.load_model(modeldir)

        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]
        classifier_filename_exp = os.path.expanduser(classifier_filename)
        with open(classifier_filename_exp, 'rb') as infile:
            (model, class_names) = pickle.load(infile)

        # video_capture = cv2.VideoCapture("akshay_mov.mp4") 


def predict(image1):
        global sess,pnet,rnet,onet,embedding_size,images_placeholder,embeddings,model,HumanNames,phase_train_placeholder
      
        minsize = 20  # minimum size of face
        threshold = [0.6, 0.7, 0.7]  # three steps's threshold
        factor = 0.709  # scale factor
        margin = 44
        frame_interval = 3
        batch_size = 1000
        image_size = 182
        input_image_size = 160
        c = 0
 
        prevTime = 0
        # ret, frame = video_capture.read()

        curTime = time.time()+1    # calc fps
        timeF = frame_interval

        if (c % timeF == 0):
            find_results = []
            emb_array = np.zeros((1, embedding_size))
            # frame = frame[:, :, 0:3]
            # bounding_boxes, _ = detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)
            # nrof_faces = bounding_boxes.shape[0] # số detec
            # print('Face Detected: %d' % nrof_faces)
            nrof_faces =1
            if nrof_faces > 0:
                cropped = []
                scaled = []
                scaled_reshape = []
                i=0
                if(True):
                    cropped.append(image1)
                    cropped[i] = facenet.flip(cropped[i], False)
                    scaled.append(misc.imresize(cropped[i], (image_size, image_size), interp='bilinear'))
                    scaled[i] = cv2.resize(scaled[i], (input_image_size,input_image_size),
                                           interpolation=cv2.INTER_CUBIC)
                    scaled[i] = facenet.prewhiten(scaled[i])
                    scaled_reshape.append(scaled[i].reshape(-1,input_image_size,input_image_size,3))
                    feed_dict = {images_placeholder: scaled_reshape[i], phase_train_placeholder: False}
                    emb_array[0, :] = sess.run(embeddings, feed_dict=feed_dict)
                    predictions = model.predict_proba(emb_array )
                    best_class_indices = np.argmax(predictions, axis=1)
          return HumanNames[best_class_indices[0]]
                    


  

In [8]:
Init()

Loading feature extraction model
Model filename: model/20170511-185253.pb


In [10]:
predict(cv2.imread("pre_img/hao/3.png"))

hao


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
